<a href="https://colab.research.google.com/github/liuyao12/Ranger-Mish-ImageWoof-5/blob/master/Woof_128_ConvTwist_separable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ResNet with a Twist

> ConvTwist + Ranger + Mish + MaxBlurPool + restrick

# setup and imports

In [ ]:
# pip install git+https://github.com/kornia/kornia

In [2]:
pip install git+https://github.com/ayasyrev/model_constructor

  Cloning https://github.com/ayasyrev/model_constructor to /tmp/pip-req-build-r2ttehgz
  Running command git clone -q https://github.com/ayasyrev/model_constructor /tmp/pip-req-build-r2ttehgz
  Created wheel for model-constructor: filename=model_constructor-0.1.1-cp36-none-any.whl size=23583 sha256=b88666ac2eb592d01f7bdbed7d01e8045027126ec67111d9de32842541aad990
  Stored in directory: /tmp/pip-ephem-wheel-cache-myf2luqk/wheels/5b/92/65/8093a307d5802f41f4a8776b40bc12b558e75a2a906ae8b683
Successfully built model-constructor


In [3]:
pip install git+https://github.com/ayasyrev/imagenette_experiments

  Cloning https://github.com/ayasyrev/imagenette_experiments to /tmp/pip-req-build-in_1bx9w
  Running command git clone -q https://github.com/ayasyrev/imagenette_experiments /tmp/pip-req-build-in_1bx9w
     |████████████████████████████████| 163kB 9.4MB/s 
  Created wheel for imagenette-experiments: filename=imagenette_experiments-0.0.1-cp36-none-any.whl size=15332 sha256=7a270e211239031a3a0eff8dae7a10f3ea2e20b304e9b6a844f29b7b5a22a88b
  Stored in directory: /tmp/pip-ephem-wheel-cache-8i1jqigl/wheels/af/99/98/2212941f45a18bf6d191f56c39e3569138414560c76defc0d4
Successfully built imagenette-experiments
ERROR: kornia 0.3.1 has requirement torch==1.5.0, but you'll have torch 1.5.1+cu101 which is incompatible.


In [4]:
from imagenette_experiments.train_utils import *

In [5]:
from kornia.contrib import MaxBlurPool2d

In [6]:
from fastai.basic_train import *
from fastai.vision import *
# from fastai.script import *
from model_constructor.net import Net, act_fn
from model_constructor.layers import SimpleSelfAttention, ConvLayer

# Twist

In [12]:
class ConvTwist(nn.Module):  # replacing 3x3 Conv2d with in_channels=out_channels
    def __init__(self, ni, nf):
        super(ConvTwist, self).__init__()
        self.twist = True # and n>=128
        self.expansion = 1
        ni *= self.expansion*2
        self.conv = nn.Conv2d(ni, ni, kernel_size=3, padding=1, bias=False, groups=1)
        if self.twist:
            Dx = torch.Tensor([[-1,0,1],[-2,0,2],[-1,0,1]]).view(1,1,3,3)
            Dy = torch.Tensor([[1,2,1],[0,0,0],[-1,-2,-1]]).view(1,1,3,3)
            self.DD = torch.cat([Dx,Dy]*(ni//2), dim=0)
            self.conv1x1 = nn.Conv2d(ni, ni, kernel_size=1, bias=False, groups=1)
            self.zero_out = torch.zeros((ni,ni,1,1))
            for i in range(ni//2):
                self.zero_out[i*2:i*2+2] = 1
        self.XY = None

    def forward(self, x):
        N,C,H,W = x.size()
        CC = C*self.expansion*2
        x = x.unsqueeze(2).expand(N,C,CC//C,H,W).reshape(N,CC,H,W)
        # x = torch.cat([x]*self.expansion, dim=1)
        if self.twist:
            if self.XY is None:
                XX = torch.from_numpy(np.indices((1,1,H,W))[3]*2/W-1).type(x.dtype).to(x.device)
                YY = torch.from_numpy(np.indices((1,1,H,W))[2]*2/H-1).type(x.dtype).to(x.device)
                self.XY = torch.cat([XX,YY]*(CC//2), dim=1)
                print(self.XY.size())
                self.DD = self.DD.to(x.device)
                self.zero_out = self.zero_out.to(x.device)
            mask = F.relu(x)+0.01
            M = mask.sum((2,3))
            XYbar = (self.XY*mask).sum((2,3))/M
            XY = self.XY-XYbar[:,:,None,None]
            # self.conv1x1.weight.data = self.conv1x1.weight * self.zero_out
            for _ in range(1):
                x0 = F.conv2d(x, self.DD, padding=1, groups=CC)
                x = self.conv(x) + XY * self.conv1x1(x0)
        return x

In [ ]:
A = torch.arange(12).view(3,4)
print(A.size())
A.unsqueeze_(1)
print(A.size())
A = A.expand((3,2,4)).reshape(6,4)
print(A)

torch.Size([3, 4])
torch.Size([3, 1, 4])
tensor([[ 0,  1,  2,  3],
        [ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [ 8,  9, 10, 11]])


# ResBlock

In [13]:
class NewLayer(nn.Sequential):
    """Basic conv layers block"""
    def __init__(self, ni, nf, ks=3, stride=1,
            act=True,  act_fn=nn.ReLU(inplace=True),
            bn_layer=True, bn_1st=True, zero_bn=False,
            padding=None, bias=False, groups=1, **kwargs):

        if padding==None: padding = ks//2
        if ni%nf==0 and ks==3:  layers = [('ConvTwist', ConvTwist(ni,nf))]
        else: layers = [('Conv{}x{}'.format(ks,ks), 
                  nn.Conv2d(ni, nf, ks, stride=stride, padding=padding, bias=bias, groups=groups))]

        act_bn = [('act_fn', act_fn)] if act else []
        if bn_layer:
            bn = nn.BatchNorm2d(nf)
            nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
            act_bn += [('bn', bn)]
        if bn_1st: act_bn.reverse()
        layers += act_bn
        super().__init__(OrderedDict(layers))

In [14]:
class NewResBlock(Module):
    def __init__(self, expansion, ni, nh, stride=1,
                 conv_layer=ConvLayer, act_fn=act_fn, bn_1st=True,
                 pool=nn.AvgPool2d(2, ceil_mode=True), sa=False, sym=False, zero_bn=True):
        nf,ni = nh*expansion,ni*expansion
        conv_layer = NewLayer
        self.reduce = noop if stride==1 else pool
        layers  = [(f"conv_0", conv_layer(ni, nh, 3, act_fn=act_fn, bn_1st=bn_1st)),
                   (f"conv_1", conv_layer(nh, nf, 3, zero_bn=zero_bn, act=False, bn_1st=bn_1st))
        ] if expansion == 1 else [
                   (f"conv_0", conv_layer(ni, nh, 1, act_fn=act_fn, bn_1st=bn_1st)),
                   (f"conv_1", conv_layer(nh, nh*2, 3, act_fn=act_fn, bn_1st=bn_1st)),
                   (f"conv_2", conv_layer(nh*2, nf, 1, zero_bn=zero_bn, act=False, bn_1st=bn_1st))
        ]
        if sa: layers.append(('sa', SimpleSelfAttention(nf,ks=1,sym=sym)))
        self.convs = nn.Sequential(OrderedDict(layers))
        self.idconv = noop if ni==nf else conv_layer(ni, nf, 1, act=False, bn_1st=bn_1st)
        self.merge = act_fn

    def forward(self, x):
        o = self.reduce(x)
        return self.merge(self.convs(o) + self.idconv(o))

# Model Constructor

In [15]:
model = Net(c_out=10, layers=[3,6,8,3], expansion=4)
model.block = NewResBlock
# model.conv_layer = NewLayer # for the stem
pool = MaxBlurPool2d(3, True)
model.pool = pool
model.stem_pool = pool
model.stem_sizes = [3,32,64,64]
model.act_fn = Mish()
model.sa = True
res = []
# epochs = [20]

# Runs and results



In [ ]:
epochs = [5, 20, 80]
for e in epochs:
    mixup=0 if e<20 else 0.2
    learn = get_learn(model=model, size=192, bs=32, mixup=mixup)
    learn.fit_fc(e, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/u

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.876550,1.844522,0.414864,0.889539,02:03
1,1.614874,1.536637,0.553067,0.930008,02:03
2,1.429234,1.319937,0.646984,0.958005,02:03
3,1.278281,1.195572,0.721049,0.962586,02:03
4,1.097350,1.040642,0.786969,0.977602,02:03


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


...

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/u

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,2.027768,1.784691,0.418173,0.879868,02:03
1,1.846209,1.643852,0.491728,0.914482,02:03
2,1.704530,1.508050,0.578264,0.939425,02:03
3,1.611667,1.402039,0.611861,0.946297,02:03
4,1.513891,1.238260,0.691270,0.958005,02:03
5,1.428799,1.150261,0.735302,0.966149,02:03
6,1.399096,1.099409,0.751082,0.971240,02:03
7,1.354039,1.066069,0.775006,0.974548,02:03
8,1.307126,1.034034,0.790277,0.974039,02:06


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


Buffered data was truncated after reaching the output size limit.

conv: 1-to-1, conv1x1: fully

in 5, 20, and 80 epochs:
`0.785442, 0.882922, 0.901502`
(2.06 min per epoch)

conv: fully, conv1x1: fully

in 5, 20, and 80 epochs: `0.781878, 0.882922, 0.895393`
(2.06 min per epoch)

conv: fully, conv1x1: 2-to-2

in 5, 20, and 80 epochs:
(2.06 min per epoch)



In [48]:
for i in range(80):
    print('epoch {} {}'.format(i, learn.recorder.metrics[i][0].item()))

epoch 0 0.4319165050983429
epoch 1 0.5347416400909424
epoch 2 0.620768666267395
epoch 3 0.6877067685127258
epoch 4 0.7164673209190369
epoch 5 0.7439551949501038
epoch 6 0.7755153775215149
epoch 7 0.7767879962921143
epoch 8 0.8055484890937805
epoch 9 0.7940952181816101
epoch 10 0.8088572025299072
epoch 11 0.8136930465698242
epoch 12 0.8114023804664612
epoch 13 0.8294731378555298
epoch 14 0.8266734480857849
epoch 15 0.8406718969345093
epoch 16 0.8271824717521667
epoch 17 0.8304912447929382
epoch 18 0.8539068698883057
epoch 19 0.8437261581420898
epoch 20 0.8554339408874512
epoch 21 0.8569610714912415
epoch 22 0.8490710258483887
epoch 23 0.8485620021820068
epoch 24 0.8523797392845154
epoch 25 0.8620514273643494
epoch 26 0.8447442054748535
epoch 27 0.8493255376815796
epoch 28 0.8617969155311584
epoch 29 0.8513616919517517
epoch 30 0.8689233660697937
epoch 31 0.8617969155311584
epoch 32 0.855942964553833
epoch 33 0.86790531873703
epoch 34 0.8712140321731567
epoch 35 0.8648511171340942
epoch 

In [54]:
for i in range(80):
    print('epoch {} {}'.format(i, learn.recorder.metrics[i][0].item()))

epoch 0 0.39373886585235596
epoch 1 0.5291422605514526
epoch 2 0.6054975986480713
epoch 3 0.6536014080047607
epoch 4 0.6759989857673645
epoch 5 0.7179943919181824
epoch 6 0.7459913492202759
epoch 7 0.7755153775215149
epoch 8 0.7806057333946228
epoch 9 0.8131840229034424
epoch 10 0.814202070236206
epoch 11 0.8157292008399963
epoch 12 0.8022397756576538
epoch 13 0.8228557109832764
epoch 14 0.8256554007530212
epoch 15 0.8416900038719177
epoch 16 0.8315092921257019
epoch 17 0.8355816006660461
epoch 18 0.8437261581420898
epoch 19 0.8378722071647644
epoch 20 0.8460168242454529
epoch 21 0.8393993377685547
epoch 22 0.8513616919517517
epoch 23 0.8485620021820068
epoch 24 0.8480529189109802
epoch 25 0.8567065596580505
epoch 26 0.8511071801185608
epoch 27 0.8556884527206421
epoch 28 0.8498345613479614
epoch 29 0.853397786617279
epoch 30 0.8612878322601318
epoch 31 0.8569610714912415
epoch 32 0.8605242967605591
epoch 33 0.8615424036979675
epoch 34 0.8518707156181335
epoch 35 0.858488142490387
epoc